In [1]:
#Import Statements
import deepchem as dc
from rdkit import Chem
from rdkit.Chem import Draw
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, losses

import pandas as pd

from matplotlib import pyplot as plt

import numpy as np

from sklearn import metrics
from sklearn.metrics import f1_score


import keras
from keras import layers

In [2]:
#GraphConv featuriser - splitting the training dataset into smaller datasets
tasks, datasets, transformers = dc.molnet.load_tox21(
    featurizer = dc.feat.ConvMolFeaturizer(),
    save_dir=r'C:\Users\ym20201\Documents\Datasets',
    data_dir=r'C:\Users\ym20201\Documents\Datasets')

# featurizer = dc.feat.ConvMolFeaturizer()
splitter = dc.splits.RandomSplitter()

# train_data, valid_data,test_data = splitter.train_valid_test_split(
#     datasets[0], 
#     frac_train = 0.8, frac_valid = 0.08, frac_test = 0.12)

train_data, valid_data,test_data = datasets


In [32]:
train_data.X

array([<deepchem.feat.mol_graphs.ConvMol object at 0x0000017F4EB06B08>,
       ...,
      dtype=object)

### Graph Convolutional NN

In [40]:
convmol_model = dc.models.GraphConvModel(
    n_tasks = len(tasks),
    graph_conv_layers=[64,64],
    dense_layer_size=200,
    dropout=0.2,
    mode='classification',
    n_classes=12,
    learning_rate=0.01,
    batch_size=100)

In [41]:
convmol_model.fit(
    train_data,
    nb_epoch=10)

C:\Users\ym20201\Anaconda3\envs\tdaf-tf2p7\lib\site-packages\tensorflow\python\framework\indexed_slices.py:450: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_14/graph_pool_28/Reshape_14:0", shape=(413,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_14/graph_pool_28/Reshape_13:0", shape=(413, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_14/graph_pool_28/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
C:\Users\ym20201\Anaconda3\envs\tdaf-tf2p7\lib\site-packages\tensorflow\python\framework\indexed_slices.py:450: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_14/graph_pool_28/Reshape_17:0", shape=(1366,), dtype=int32), values=Tensor("gradient_

C:\Users\ym20201\Anaconda3\envs\tdaf-tf2p7\lib\site-packages\tensorflow\python\framework\indexed_slices.py:450: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_14/graph_pool_27/Reshape_14:0", shape=(413,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_14/graph_pool_27/Reshape_13:0", shape=(413, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_14/graph_pool_27/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
C:\Users\ym20201\Anaconda3\envs\tdaf-tf2p7\lib\site-packages\tensorflow\python\framework\indexed_slices.py:450: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_14/graph_pool_27/Reshape_17:0", shape=(1366,), dtype=int32), values=Tensor("gradient_

C:\Users\ym20201\Anaconda3\envs\tdaf-tf2p7\lib\site-packages\tensorflow\python\framework\indexed_slices.py:450: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_14/graph_pool_28/Reshape_14:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_14/graph_pool_28/Reshape_13:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_14/graph_pool_28/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
C:\Users\ym20201\Anaconda3\envs\tdaf-tf2p7\lib\site-packages\tensorflow\python\framework\indexed_slices.py:450: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_14/graph_pool_28/Reshape_17:0", shape=(None,), dtype=int32), values=Tensor("gradien

C:\Users\ym20201\Anaconda3\envs\tdaf-tf2p7\lib\site-packages\tensorflow\python\framework\indexed_slices.py:450: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_14/graph_pool_28/Reshape_26:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_14/graph_pool_28/Reshape_25:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_14/graph_pool_28/Cast_8:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
C:\Users\ym20201\Anaconda3\envs\tdaf-tf2p7\lib\site-packages\tensorflow\python\framework\indexed_slices.py:450: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_14/graph_pool_28/Reshape_29:0", shape=(None,), dtype=int32), values=Tensor("gradien

0.9747229894002278

In [42]:
# metric = dc.metrics.Metric(dc.metrics.roc_auc_score) #doesnt work

#Accuracy classification score
metric = dc.metrics.Metric(dc.metrics.balanced_accuracy_score)
convmol_model.evaluate(train_data, [metric], transformers)

{'balanced_accuracy_score': 0.7405171117875738}

In [69]:
train_data.y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [74]:
pred = convmol_model.predict(train_data)
pred

array([[[8.33617210e-01, 1.63484052e-01, 4.67835169e-04, ...,
         8.67635972e-05, 1.31598790e-04, 2.05754550e-04],
        [9.24356461e-01, 7.25726634e-02, 4.70698906e-05, ...,
         3.05094494e-04, 1.89511018e-04, 8.32762526e-05],
        [5.87989032e-01, 4.11606967e-01, 2.73202531e-05, ...,
         5.92077558e-05, 2.04156895e-05, 6.85867917e-06],
        ...,
        [6.08100414e-01, 3.91219854e-01, 7.43473720e-05, ...,
         2.65122799e-05, 4.07190746e-05, 3.25317633e-05],
        [7.48789966e-01, 2.50528634e-01, 3.64298903e-05, ...,
         6.71951348e-05, 7.59222094e-05, 7.30786705e-05],
        [8.11415970e-01, 1.88235104e-01, 6.19808816e-06, ...,
         6.78201977e-05, 2.98236610e-05, 4.48059400e-05]],

       [[9.84343529e-01, 1.12059331e-02, 3.50672839e-04, ...,
         1.75566573e-04, 2.02706084e-04, 1.54363675e-04],
        [9.83691812e-01, 1.53255565e-02, 2.58297350e-05, ...,
         5.48934186e-05, 1.27633699e-04, 5.60281114e-05],
        [9.77670789e-01, 

In [8]:
# #epoch - number of times data is shown to NN
# model.fit(train_data, nb_epoch=10) #training model

In [12]:
# #Converting numpy array to tensor
# train_tensor = []
# for i in range(len(train_feat)):
#     tensor = tf.convert_to_tensor(train_feat[i])
#     train_tensor.append(tensor)
    
# valid_tensor = []
# for i in range(len(valid_feat)):
#     tensor_v = tf.convert_to_tensor(valid_feat[i])
#     valid_tensor.append(tensor_v)
    
# test_tensor = []
# for i in range(len(test_feat)):
#     tensor_t = tf.convert_to_tensor(test_feat[i])
#     test_tensor.append(tensor_t)

In [22]:
# metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
# print("Training set score:", model.evaluate(train_data, [metric], transformers))
# print("Test set score:", model.evaluate(test_data, [metric], transformers))

In [13]:
# #Constructing the model
# # keras_model = Sequential([
# #     layers.Dense(50, activation='relu'), # hidden layer
# #     layers.Dense(1) # output layer
# # ])

# # model = dc.models.KerasModel(keras_model, dc.models.losses.L1Loss())
# # model.fit(train_feat, nb_epoch=10)

# keras_model = Sequential()
# keras_model.add(layers.Conv2D(filters=32,
#                           kernel_size=3,
#                           activation='relu',
#                           dilation_rate=2,
#                              input_shape=(80, 80, 1)))
# keras_model.add(layers.MaxPooling2D(pool_size=(2,2)))
# #dropout to avoid overfitting
# keras_model.add(layers.Dense(50, activation='relu'))
# keras_model.add(layers.Dense(1))

# keras_model.build()

In [14]:
# keras_model.compile(optimizer='adam', loss='mean_squared_error')
# keras_model.summary()

In [45]:
# tf.convert_to_tensor(train_feat[0])


In [15]:
# keras_model.fit(x=train_feat,
#                 y=train_feat,
#                 epochs=10,
#                 batch_size=512,
#                 shuffle=True,
#                validation_data=(valid_feat, valid_feat))

In [12]:
#Autoencoder